In [1]:
import ssl
import nltk
import certifi
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [2]:
# Load the pre-trained model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create a sentiment-analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [3]:
sentiment_analyzer('Ursula')

[{'label': 'LABEL_1', 'score': 0.6010197997093201}]

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving final_test_all.csv to final_test_all.csv
User uploaded file "final_test_all.csv" with length 147833 bytes


In [5]:
conversations = pd.read_csv("final_test_all.csv")
conversations.head(10)

,1,Unnamed: 1,actual_sent,text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,2,"1,13118E+18",negative,Forcibly took my daughter's bag from her to ch...,NaN,en,1604219274,78,14,17,42,520,1,NaN,"1,13118E+18",NaN
1,3,"1,13119E+18",positive,@addblackman We understand that you would pref...,1604219274,en,56377143,279,47,2336092,65146,641,6762,"21,88333333","1,13118E+18",NaN
2,4,"1,13119E+18",positive,@addblackman We do understand that some passen...,1604219274,en,56377143,274,43,2336092,65146,641,6762,"22,31666667","1,13118E+18",NaN
3,5,"1,13119E+18",neutral,"@addblackman Keys, medicines or your laptop ma...",1604219274,en,56377143,300,45,2336092,65146,641,6762,"23,16666667","1,13118E+18",NaN
4,6,"1,13119E+18",positive,"@KLM Hi, any chance you can retrieve an old bo...",56377143,en,4806232401,178,36,31,241,7188,6,NaN,"1,13119E+18",NaN
5,7,"1,1312E+18",neutral,"@stargirlTT Unfortunately, we cannot send a bo...",4806232401,en,56377143,86,13,2336094,65146,641,6762,"21,86666667","1,13119E+18",NaN
6,8,"1,1312E+18",positive,@KLM No problem. Thanks for getting back to me...,56377143,en,4806232401,58,11,31,241,7190,6,"1,5","1,13119E+18",NaN
7,9,"1,1312E+18",negative,@klm last time I fly with you. Yesterday you a...,56377143,en,15168078,273,46,99,389,14,1,NaN,"1,1312E+18",NaN
8,10,"1,1312E+18",positive,@astonbowles If all your flights are booked in...,15168078,en,56377143,239,42,2336096,65146,641,6762,"4,733333333","1,1312E+18",NaN
9,11,"1,13122E+18",negative,@KLM The next available flight is 24hrs later ...,56377143,en,15168078,126,21,99,389,14,1,"59,18333333","1,1312E+18",NaN


In [6]:
# Configure SSL context to use certifis CA bundle
ssl_context = ssl.create_default_context(cafile=certifi.where())
ssl._create_default_https_context = lambda: ssl_context

# Download the vader_lexicon data
nltk.download('vader_lexicon')

analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [7]:
# Data Cleaning (only remove URLs)
# Overrides the existing column "cleaned_text"

# Function to clean tweet text without removing special characters
def clean_tweet_text(text):
    # Remove URLs
    text = str(text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    return text

# Apply cleaning function to the text column
conversations['clean_text'] = conversations['text'].apply(clean_tweet_text)

# conversations = conversations.drop(conversations[conversations.sentiment == "Irrelevant"].index)

conversations.head(10)

,1,Unnamed: 1,actual_sent,text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,clean_text
0,2,"1,13118E+18",negative,Forcibly took my daughter's bag from her to ch...,NaN,en,1604219274,78,14,17,42,520,1,NaN,"1,13118E+18",NaN,Forcibly took my daughter's bag from her to ch...
1,3,"1,13119E+18",positive,@addblackman We understand that you would pref...,1604219274,en,56377143,279,47,2336092,65146,641,6762,"21,88333333","1,13118E+18",NaN,@addblackman We understand that you would pref...
2,4,"1,13119E+18",positive,@addblackman We do understand that some passen...,1604219274,en,56377143,274,43,2336092,65146,641,6762,"22,31666667","1,13118E+18",NaN,@addblackman We do understand that some passen...
3,5,"1,13119E+18",neutral,"@addblackman Keys, medicines or your laptop ma...",1604219274,en,56377143,300,45,2336092,65146,641,6762,"23,16666667","1,13118E+18",NaN,"@addblackman Keys, medicines or your laptop ma..."
4,6,"1,13119E+18",positive,"@KLM Hi, any chance you can retrieve an old bo...",56377143,en,4806232401,178,36,31,241,7188,6,NaN,"1,13119E+18",NaN,"@KLM Hi, any chance you can retrieve an old bo..."
5,7,"1,1312E+18",neutral,"@stargirlTT Unfortunately, we cannot send a bo...",4806232401,en,56377143,86,13,2336094,65146,641,6762,"21,86666667","1,13119E+18",NaN,"@stargirlTT Unfortunately, we cannot send a bo..."
6,8,"1,1312E+18",positive,@KLM No problem. Thanks for getting back to me...,56377143,en,4806232401,58,11,31,241,7190,6,"1,5","1,13119E+18",NaN,@KLM No problem. Thanks for getting back to me...
7,9,"1,1312E+18",negative,@klm last time I fly with you. Yesterday you a...,56377143,en,15168078,273,46,99,389,14,1,NaN,"1,1312E+18",NaN,@klm last time I fly with you. Yesterday you a...
8,10,"1,1312E+18",positive,@astonbowles If all your flights are booked in...,15168078,en,56377143,239,42,2336096,65146,641,6762,"4,733333333","1,1312E+18",NaN,@astonbowles If all your flights are booked in...
9,11,"1,13122E+18",negative,@KLM The next available flight is 24hrs later ...,56377143,en,15168078,126,21,99,389,14,1,"59,18333333","1,1312E+18",NaN,@KLM The next available flight is 24hrs later ...


In [8]:
conversations['compound'] = [analyser.polarity_scores(x)['compound'] for x in conversations['clean_text']]

In [9]:
conversations.head(10)

,1,Unnamed: 1,actual_sent,text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,clean_text,compound
0,2,"1,13118E+18",negative,Forcibly took my daughter's bag from her to ch...,NaN,en,1604219274,78,14,17,42,520,1,NaN,"1,13118E+18",NaN,Forcibly took my daughter's bag from her to ch...,0.0000
1,3,"1,13119E+18",positive,@addblackman We understand that you would pref...,1604219274,en,56377143,279,47,2336092,65146,641,6762,"21,88333333","1,13118E+18",NaN,@addblackman We understand that you would pref...,0.7506
2,4,"1,13119E+18",positive,@addblackman We do understand that some passen...,1604219274,en,56377143,274,43,2336092,65146,641,6762,"22,31666667","1,13118E+18",NaN,@addblackman We do understand that some passen...,-0.6619
3,5,"1,13119E+18",neutral,"@addblackman Keys, medicines or your laptop ma...",1604219274,en,56377143,300,45,2336092,65146,641,6762,"23,16666667","1,13118E+18",NaN,"@addblackman Keys, medicines or your laptop ma...",0.6705
4,6,"1,13119E+18",positive,"@KLM Hi, any chance you can retrieve an old bo...",56377143,en,4806232401,178,36,31,241,7188,6,NaN,"1,13119E+18",NaN,"@KLM Hi, any chance you can retrieve an old bo...",0.2500
5,7,"1,1312E+18",neutral,"@stargirlTT Unfortunately, we cannot send a bo...",4806232401,en,56377143,86,13,2336094,65146,641,6762,"21,86666667","1,13119E+18",NaN,"@stargirlTT Unfortunately, we cannot send a bo...",-0.3400
6,8,"1,1312E+18",positive,@KLM No problem. Thanks for getting back to me...,56377143,en,4806232401,58,11,31,241,7190,6,"1,5","1,13119E+18",NaN,@KLM No problem. Thanks for getting back to me...,0.2183
7,9,"1,1312E+18",negative,@klm last time I fly with you. Yesterday you a...,56377143,en,15168078,273,46,99,389,14,1,NaN,"1,1312E+18",NaN,@klm last time I fly with you. Yesterday you a...,-0.7096
8,10,"1,1312E+18",positive,@astonbowles If all your flights are booked in...,15168078,en,56377143,239,42,2336096,65146,641,6762,"4,733333333","1,1312E+18",NaN,@astonbowles If all your flights are booked in...,-0.5574
9,11,"1,13122E+18",negative,@KLM The next available flight is 24hrs later ...,56377143,en,15168078,126,21,99,389,14,1,"59,18333333","1,1312E+18",NaN,@KLM The next available flight is 24hrs later ...,-0.4767


In [10]:
# Function to categorize sentiment based on score
def categorize_sentiment(row):
    score = row["compound"]

    if score > 0.05:
        return "positive"
    elif score < -0.05:
        return "negative"
    else:
        return "neutral"

# # Apply categorization function to the sentiment score column
conversations['vader'] = conversations.apply(categorize_sentiment, axis=1)

def is_equal(row):
    sentiment = row["actual_sent"]
    vader = row["vader"]

    if vader == sentiment:
        return True
    else:
        return False

conversations["vader_acc"] = conversations.apply(is_equal, axis=1)

In [11]:
conversations.head(20)

counts = conversations['vader_acc'].value_counts()
print(counts)

vader_acc
True     332
False    246
Name: count, dtype: int64


In [12]:
conversations.head(10)

,1,Unnamed: 1,actual_sent,text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,clean_text,compound,vader,vader_acc
0,2,"1,13118E+18",negative,Forcibly took my daughter's bag from her to ch...,NaN,en,1604219274,78,14,17,42,520,1,NaN,"1,13118E+18",NaN,Forcibly took my daughter's bag from her to ch...,0.0000,neutral,False
1,3,"1,13119E+18",positive,@addblackman We understand that you would pref...,1604219274,en,56377143,279,47,2336092,65146,641,6762,"21,88333333","1,13118E+18",NaN,@addblackman We understand that you would pref...,0.7506,positive,True
2,4,"1,13119E+18",positive,@addblackman We do understand that some passen...,1604219274,en,56377143,274,43,2336092,65146,641,6762,"22,31666667","1,13118E+18",NaN,@addblackman We do understand that some passen...,-0.6619,negative,False
3,5,"1,13119E+18",neutral,"@addblackman Keys, medicines or your laptop ma...",1604219274,en,56377143,300,45,2336092,65146,641,6762,"23,16666667","1,13118E+18",NaN,"@addblackman Keys, medicines or your laptop ma...",0.6705,positive,False
4,6,"1,13119E+18",positive,"@KLM Hi, any chance you can retrieve an old bo...",56377143,en,4806232401,178,36,31,241,7188,6,NaN,"1,13119E+18",NaN,"@KLM Hi, any chance you can retrieve an old bo...",0.2500,positive,True
5,7,"1,1312E+18",neutral,"@stargirlTT Unfortunately, we cannot send a bo...",4806232401,en,56377143,86,13,2336094,65146,641,6762,"21,86666667","1,13119E+18",NaN,"@stargirlTT Unfortunately, we cannot send a bo...",-0.3400,negative,False
6,8,"1,1312E+18",positive,@KLM No problem. Thanks for getting back to me...,56377143,en,4806232401,58,11,31,241,7190,6,"1,5","1,13119E+18",NaN,@KLM No problem. Thanks for getting back to me...,0.2183,positive,True
7,9,"1,1312E+18",negative,@klm last time I fly with you. Yesterday you a...,56377143,en,15168078,273,46,99,389,14,1,NaN,"1,1312E+18",NaN,@klm last time I fly with you. Yesterday you a...,-0.7096,negative,True
8,10,"1,1312E+18",positive,@astonbowles If all your flights are booked in...,15168078,en,56377143,239,42,2336096,65146,641,6762,"4,733333333","1,1312E+18",NaN,@astonbowles If all your flights are booked in...,-0.5574,negative,False
9,11,"1,13122E+18",negative,@KLM The next available flight is 24hrs later ...,56377143,en,15168078,126,21,99,389,14,1,"59,18333333","1,1312E+18",NaN,@KLM The next available flight is 24hrs later ...,-0.4767,negative,True


In [ ]:
conversations.shape

(111, 15)